In [1]:
import logging
import sys
import time

import flimlib
import napari
from napari.qt.threading import thread_worker
import numpy as np
import h5py


class ListArray:
    """An array-like object backed by a list of ndarrays."""

    def __init__(self, arrays):
        if not len(arrays):
            raise ValueError # At least for now, don't allow empty

        self._arrays = []
        self._dtype = None
        self._shape = None
        for a in arrays:
            self._arrays.append(np.asarray(a))
            if self._dtype is None:
                self._dtype = self._arrays[0].dtype
            elif self._arrays[-1].dtype != self._dtype:
                raise ValueError
            if self._shape is None:
                self._shape = self._arrays[0].shape
            elif self._arrays[-1].shape != self._shape:
                raise ValueError

    @property
    def ndim(self):
        return 1 + len(self._shape)

    @property
    def dtype(self):
        return self._dtype

    @property
    def shape(self):
        return (len(self._arrays),) + self._shape

    def __getitem__(self, slices):
        if not isinstance(slices, tuple):
            slices = (slices,)
        ndslices = slices[1:]
        s0 = slices[0]
        if isinstance(s0, slice):
            start, stop, step = s0.indices(len(self._arrays))
            dim0 = (stop - start) // step
            shape = (dim0,) + self._shape
            ret = np.empty_like(self._arrays[0], shape=shape)
            j0 = 0
            for i0 in range(start, stop, step):
                ret[j0] = self._arrays[i0][ndslices]
                j0 += 1
            return ret
        else:  # s0 is an integer
            return self._arrays[s0][ndslices]

    def __array__(self):
        return self[:]

In [2]:
############ Copied from napari/_vispy/cameras.py ############
from vispy.scene.cameras import PanZoomCamera
import numpy as np


class PanZoom1DCamera(PanZoomCamera):
    def __init__(self, axis=1, *args, **kwargs):
        """A camera that can only Pan/Zoom along one axis.
        Useful in a PlotWidget.
        Parameters
        ----------
        axis : int, optional
            The axis to constrain. (This axis will NOT pan or zoom).
                0 => lock x axis
                1 => lock y axis
                by default 1
        """
        self.axis = axis
        super().__init__(*args, **kwargs)

    def zoom(self, factor, center=None):
        if np.isscalar(factor):
            factor = [factor, factor]
        factor[self.axis] = 1
        return super().zoom(factor, center=center)

    def pan(self, pan):
        pan[self.axis] = 0
        self.rect = self.rect + pan


############ Copied from napari/_vispy/vispy_plot.py ############
import numpy as np
from vispy import scene
from vispy.plot import PlotWidget as VispyPlotWidget



# PlotWidget inherits from scene.Widget
class PlotWidget(VispyPlotWidget):
    """Subclass of vispy.plot.PlotWidget.
    Subclassing mostly to override styles (which are not exposed in the main
    class) and to override `_configure_2d` to allow more control over the
    layout.
    """

    # default styles to use for the AxisVisuals
    AXIS_KWARGS = {
        'text_color': 'w',
        'axis_color': 'w',
        'tick_color': 'w',
        'tick_width': 1,
        'tick_font_size': 8,
        'tick_label_margin': 12,
        'axis_label_margin': 50,
        'minor_tick_length': 2,
        'major_tick_length': 5,
        'axis_width': 1,
        'axis_font_size': 9,
    }
    # default styles to use for the title
    TITLE_KWARGS = {'font_size': 16, 'color': 'w'}

    def __init__(
        self,
        margin=10,
        fg_color=None,
        xlabel="",
        ylabel="",
        title="",
        show_yaxis=True,
        show_xaxis=True,
        cbar_length=50,
        axis_kwargs=None,
        title_kwargs=None,
        lock_axis=None,
        **kwargs,
    ):
        self._fg = fg_color
        self.grid = None
        self.camera = None
        self.title = None
        self.title_widget = None
        self.xaxis = None
        self.yaxis = None
        self.xaxis_widget = None
        self.yaxis_widget = None
        self.show_xaxis = show_xaxis
        self.show_yaxis = show_yaxis
        self.xlabel = scene.Label(str(xlabel), color=fg_color or '#ccc')
        self.ylabel = scene.Label(
            str(ylabel), rotation=-90, color=fg_color or '#ccc'
        )
        self.ylabel_widget = None
        self.xlabel_widget = None
        self.padding_left = None
        self.padding_bottom = None
        self.padding_right = None
        self._locked_axis = lock_axis

        self.axis_kwargs = self.AXIS_KWARGS
        if isinstance(axis_kwargs, dict):
            self.axis_kwargs.update(axis_kwargs)
        elif axis_kwargs is not None:
            raise TypeError(
                f'axis_kwargs must be a dict.  got: {type(axis_kwargs)}'
            )
        if fg_color is not None:
            self.axis_kwargs['text_color'] = fg_color
            self.axis_kwargs['axis_color'] = fg_color
            self.axis_kwargs['tick_color'] = fg_color

        self._configured = False
        self.visuals = []

        self.cbar_top = None
        self.cbar_bottom = None
        self.cbar_left = None
        self.cbar_right = None
        self.cbar_length = cbar_length

        super(VispyPlotWidget, self).__init__(**kwargs)
        self.grid = self.add_grid(spacing=0, margin=margin)

        if isinstance(title_kwargs, dict):
            self.TITLE_KWARGS.update(title_kwargs)
        elif title_kwargs is not None:
            raise TypeError(
                f'axis_ktitle_kwargswargs must be a dict.  got: {type(title_kwargs)}'
            )

        self.title = scene.Label(str(title), **self.TITLE_KWARGS)

    def autoscale(self, axes='both'):
        # might be too slow?
        x, y = None, None
        for visual in self.visuals:
            data = None
            if hasattr(visual, '_line'):
                data = np.array(visual._line._pos).T
            elif hasattr(visual, '_markers'):
                data = np.array(visual._markers._data['a_position'])[:, :2].T
            if data is not None and len(data.shape):
                if axes in ('y', 'both'):
                    y = y if y is not None else [np.inf, -np.inf]
                    y[1] = np.maximum(data[1].max(), y[1])
                    y[0] = np.minimum(data[1].min(), y[0])
                if axes in ('x', 'both'):
                    x = x if x is not None else [np.inf, -np.inf]
                    x[1] = np.maximum(data[0].max(), x[1])
                    x[0] = np.minimum(data[0].min(), x[0])

        self.view.camera.set_range(x, y, margin=0.005)

    def _configure_2d(self, fg_color=None):
        if self._configured:
            return

        #         c0        c1      c2      c3      c4      c5         c6
        #     +---------+-------+-------+-------+-------+---------+---------+
        #  r0 |         |                       | title |         |         |
        #     |         +-----------------------+-------+---------+         |
        #  r1 |         |                       | cbar  |         |         |
        #     | ------- +-------+-------+-------+-------+---------+ ------- |
        #  r2 | padding | cbar  | ylabel| yaxis |  view | cbar    | padding |
        #     | ------- +-------+-------+-------+-------+---------+ ------- |
        #  r3 |         |                       | xaxis |         |         |
        #     |         +-----------------------+-------+---------+         |
        #  r4 |         |                       | xlabel|         |         |
        #     |         +-----------------------+-------+---------+         |
        #  r5 |         |                       | cbar  |         |         |
        #     |---------+-----------------------+-------+---------+---------|
        #  r6 |                                 |padding|                   |
        #     +---------+-----------------------+-------+---------+---------+

        # PADDING
        self.padding_right = self.grid.add_widget(None, row=2, col=6)
        self.padding_right.width_min = 1
        self.padding_right.width_max = 5
        self.padding_bottom = self.grid.add_widget(None, row=6, col=4)
        self.padding_bottom.height_min = 1
        self.padding_bottom.height_max = 3

        # TITLE
        self.title_widget = self.grid.add_widget(self.title, row=0, col=4)
        self.title_widget.height_min = self.title_widget.height_max = (
            30 if self.title.text else 5
        )

        # COLORBARS
        self.cbar_top = self.grid.add_widget(None, row=1, col=4)
        self.cbar_top.height_max = 0
        self.cbar_left = self.grid.add_widget(None, row=2, col=1)
        self.cbar_left.width_max = 0
        self.cbar_right = self.grid.add_widget(None, row=2, col=5)
        self.cbar_right.width_max = 0
        self.cbar_bottom = self.grid.add_widget(None, row=5, col=4)
        self.cbar_bottom.height_max = 0

        # Y AXIS
        self.yaxis = scene.AxisWidget(orientation='left', **self.axis_kwargs)
        self.yaxis_widget = self.grid.add_widget(self.yaxis, row=2, col=3)
        if self.show_yaxis:
            self.yaxis_widget.width_max = 30
            self.ylabel_widget = self.grid.add_widget(
                self.ylabel, row=2, col=2
            )
            self.ylabel_widget.width_max = 30 if self.ylabel.text else 1
            self.padding_left = self.grid.add_widget(None, row=2, col=0)
            self.padding_left.width_min = 1
            self.padding_left.width_max = 10
        else:
            self.yaxis.visible = False
            self.yaxis.width_max = 1
            self.padding_left = self.grid.add_widget(
                None, row=2, col=0, col_span=3
            )
            self.padding_left.width_min = 1
            self.padding_left.width_max = 5

        # X AXIS
        self.xaxis = scene.AxisWidget(orientation='bottom', **self.axis_kwargs)
        self.xaxis_widget = self.grid.add_widget(self.xaxis, row=3, col=4)
        self.xaxis_widget.height_max = 20 if self.show_xaxis else 0
        self.xlabel_widget = self.grid.add_widget(self.xlabel, row=4, col=4)
        self.xlabel_widget.height_max = 20 if self.xlabel.text else 0

        # VIEWBOX (this has to go last, see vispy #1748)
        self.view = self.grid.add_view(
            row=2, col=4, border_color=None, bgcolor=None
        )

        self.lock_axis(self._locked_axis)
        self._configured = True
        self.xaxis.link_view(self.view)
        self.yaxis.link_view(self.view)

    def lock_axis(self, axis):
        # work in progress
        if isinstance(axis, str):
            if axis.lower() == 'x':
                axis = 0
            elif axis.lower() == 'y':
                axis = 1
            else:
                raise ValueError("axis must be either 'x' or 'y'")
        self._locked_axis = axis
        if self._locked_axis is not None:
            self.view.camera = PanZoom1DCamera(self._locked_axis)
        else:
            self.view.camera = 'panzoom'
        self.camera = self.view.camera

    def plot(
        self,
        data,
        color=(0.53, 0.56, 0.57, 1.00),
        symbol='o',
        width=1,
        marker_size=8,
        edge_color='gray',
        face_color='gray',
        edge_width=1,
        title=None,
        xlabel=None,
        ylabel=None,
        connect='strip',
    ):
        """Plot a series of data using lines and markers
        Parameters
        ----------
        data : array | two arrays
            Arguments can be passed as ``(Y,)``, ``(X, Y)`` or
            ``np.array((X, Y))``.
        color : instance of Color
            Color of the line.
        symbol : str
            Marker symbol to use.
        width : float
            Line width.
        marker_size : float
            Marker size. If `size == 0` markers will not be shown.
        edge_color : instance of Color
            Color of the marker edge.
        face_color : instance of Color
            Color of the marker face.
        edge_width : float
            Edge width of the marker.
        title : str | None
            The title string to be displayed above the plot
        xlabel : str | None
            The label to display along the bottom axis
        ylabel : str | None
            The label to display along the left axis.
        connect : str | array
            Determines which vertices are connected by lines.
        Returns
        -------
        line : instance of LinePlot
            The line plot.
        See also
        --------
        marker_types, LinePlot
        """
        self._configure_2d()
        line = scene.LinePlot(
            data,
            connect=connect,
            color=color,
            symbol=symbol,
            width=width,
            marker_size=marker_size,
            edge_color=edge_color,
            face_color=face_color,
            edge_width=edge_width,
        )
        self.view.add(line)
        self.visuals.append(line)

        if title is not None:
            self.title.text = title
        if xlabel is not None:
            self.xlabel.text = xlabel
        if ylabel is not None:
            self.ylabel.text = ylabel

        if data is not None:
            self.view.camera.set_range()
        return line

    def scatter(self, data, **kwargs):
        self._configure_2d()
        markers = scene.Markers(pos=data, **kwargs)
        self.view.add(markers)
        self.visuals.append(markers)

        if data is not None:
            self.view.camera.set_range()
        return markers


############ Copied from napari/_vispy/vispy_figure.py ############
from vispy.plot.fig import Fig as VispyFig

class Fig(VispyFig):
    """Subclas of vispy.plot.Fig mostly just to use our internal plot widget.
    """

    def __init__(
        self,
        bgcolor='k',
        size=(800, 600),
        show=True,
        keys=None,
        vsync=True,
        **kwargs,
    ):
        self._plot_widgets = []
        self._grid = None  # initialize before the freeze occurs
        super(VispyFig, self).__init__(
            bgcolor=bgcolor,
            keys=keys,
            show=show,
            size=size,
            vsync=vsync,
            **kwargs,
        )
        self._grid = self.central_widget.add_grid()
        self._grid._default_class = PlotWidget

In [16]:
global PERIOD, FIT_START, FIT_END, FONT_SIZE
PERIOD = 0.04
FIT_START = 50
FIT_END = 210
# selected_pixels has the form ((x1, x2, ...), (y1, y2, ...), ...)
SELECTED_PIXELS = ((107,107),(73,74))
FONT_SIZE = 10
TAU_MAX = 4
SELECT_RADIUS = 3
MAX_POINTS = 1

In [4]:
hf1 = h5py.File('data_ch2_med_photons.h5', 'r') # change filename
keys= [key for key in hf1.keys()]
photon_count = np.zeros(hf1.get(keys[0]).shape, dtype=np.float32)
for i in range(100):
    photon_count += hf1.get(keys[i])
mean_selected_photon_count = np.mean(photon_count[((0,),(0,))],axis=0)

In [56]:
import colorsys

def receive_and_compute(photon_count):
 
    photon_count = np.asarray(photon_count, dtype=np.float32) #does this help?
    phasor = flimlib.GCI_Phasor(PERIOD, photon_count, fit_start=FIT_START, fit_end=FIT_END)
    u = phasor.u
    v = phasor.v
    
    rld = flimlib.GCI_triple_integral_fitting_engine(PERIOD, photon_count, fit_start=FIT_START, fit_end=FIT_END)
    tau = rld.tau
    tau[tau<0] = 0
    tau[tau>TAU_MAX] = 0 # TODO how should the contrast limits be handled?

    intensity = photon_count.sum(axis=-1)
    

    return 0, 0, intensity, tau, np.asarray([u.ravel(), v.ravel()]).T
    

def compute_selected(photon_count):
    rld = flimlib.GCI_triple_integral_fitting_engine(PERIOD, photon_count, fit_start=FIT_START, fit_end=FIT_END)
    param_in = [rld.Z, rld.A, rld.tau]
    
    lm = flimlib.GCI_marquardt_fitting_engine(PERIOD, photon_count, param_in, fit_start=FIT_START, fit_end=FIT_END)
    
    return rld, lm

current_series_frames = []
# this would be yeilded
series_no, seqno, intensity, tau, phasor = receive_and_compute(photon_count)
rld_selected, lm_selected = compute_selected(mean_selected_photon_count)
intensity *= 1.0/intensity.max()
tau *= 1.0/np.nanmax(tau)
intensity_scaled_tau = np.empty([*tau.shape,3], dtype=float)

for r in range(tau.shape[0]):
    for c in range(tau.shape[1]):
        if not np.isnan(tau[r][c]):
            intensity_scaled_tau[r][c] = colorsys.hsv_to_rgb(tau[r][c], 1.0, intensity[r][c])
current_series_frames.append(intensity_scaled_tau)
arr = ListArray(current_series_frames)

In [87]:
from vispy.scene.visuals import Text
import time

try:
    if viewer:
        viewer.close()
except:
    print("viewer already closed or never opened")
    
viewer = napari.Viewer()
current_series_layer = viewer.add_image(arr)
current_series_layer.contrast_limits=(0, TAU_MAX)
viewer.layers.selection.active = current_series_layer
viewer.dims.current_step = (0,) + (0,) * tau.ndim

fig = Fig()
# add a docked figure
dw = viewer.window.add_dock_widget(fig, area='bottom')
# get a handle to the plotWidget
ax = fig[0, 0]

lm_time = np.linspace(0,(lm_selected.fitted.size-1)*PERIOD,lm_selected.fitted.size,dtype=np.float32)
rld_time = lm_time[0:rld_selected.fitted.size]

data = np.array((lm_time, mean_selected_photon_count)).T

# plot the scatter plot and line
lm_curve = ax.plot((lm_time, lm_selected.fitted), color='g', marker_size=0, width=2)
rld_curve = ax.plot((rld_time, rld_selected.fitted), color='r', marker_size=0, width=2)
data_scatter = ax.scatter(data, size=1, edge_width=0, face_color='m')
rld_info = Text('', parent=ax.view, color='r', anchor_x='right', font_size = FONT_SIZE)
lm_info = Text('', parent=ax.view, color='g', anchor_x='right', font_size = FONT_SIZE)
viewer.window._qt_window.setGeometry(400, 100, 800, 900) 

def update_plot_with_selection(selected_photon_count):
    rld_selected, lm_selected = compute_selected(selected_photon_count)
    if rld_selected.Z < 0:
        print("ERROR")
    lm_curve.set_data((lm_time, lm_selected.fitted))
    rld_curve.set_data((rld_time, rld_selected.fitted))
    data_scatter.set_data(np.array((lm_time, selected_photon_count)).T)
    rld_info.text = 'RLD | chisq = ' + "{:.2e}".format(float(rld_selected.chisq)) + ', tau = ' + "{:.2e}".format(float(rld_selected.tau))
    rld_info.pos = ax.view.size[0], rld_info.font_size
    lm_info.text = 'LMA | chisq = ' + "{:.2e}".format(float(lm_selected.chisq)) + ', tau = ' + "{:.2e}".format(float(lm_selected.param[2]))
    lm_info.pos = ax.view.size[0], rld_info.font_size*3
    ax.autoscale()

update_plot_with_selection(np.mean(photon_count[SELECTED_PIXELS],axis=0))
    

  File "C:\Users\default.LAPTOP-EIR1BCFR\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\default.LAPTOP-EIR1BCFR\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
    self.io_loop.start()
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
    self.asyncio_loop.ru

viewer already closed or never opened


Exception in callback BaseAsyncIOLoop._handle_events(14528, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(14528, 1)>
Traceback (most recent call last):
  File "C:\Users\default.LAPTOP-EIR1BCFR\AppData\Local\Programs\Python\Python39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\zmq\eventloop\zmqstream.py", line 448, in _handle_events
    self._handle_recv()
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\zmq\eventloop\zmqstream.py", line 477, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\zmq\eventloop\zmqstrea

In [88]:
"""
Example updating the status bar with line profile info while dragging
lines around in a shapes layer.
"""

import numpy as np
import napari

points_layer = viewer.add_points(
    [np.array([0,0])],
)
points_layer.mode = 'select'


# hook the points up to events
def profile_points(points_layer):
    # only a single point?
    try:
        points = []
        for point in np.round(points_layer.data).astype(int):
            #if i in points_layer._selected_data:
            
            rmax = photon_count.shape[0] - 1
            cmax = photon_count.shape[1] - 1
            for r in range(min(rmax, max(0, point[0] - SELECT_RADIUS)), min(rmax, max(0, point[0] + SELECT_RADIUS + 1))):
                for c in range(min(cmax, max(0, point[1] - SELECT_RADIUS)), min(cmax, max(0, point[1] + SELECT_RADIUS + 1))):
                    points += [[r, c]]
        if(points):
            points = tuple(np.asarray(points).T) # indexer
            update_plot_with_selection(np.mean(photon_count[points],axis=0))
    except Exception as e:
        print(e) #for debugging. remove eventually


@points_layer.mouse_drag_callbacks.append
def profile_points_drag(layer, event):
    profile_points(layer)
    yield
    while event.type == 'mouse_move':
        profile_points(layer)
        yield

def handle_new_point(event):
    global points_layer
    try:
        if points_layer.data.shape[0] > 1:
            points_layer.data = points_layer.data[1:]
            profile_points(points_layer)
    except Exception as e:
        print(e)
points_layer.events.data.connect(handle_new_point)

<function __main__.handle_new_point(event)>

  File "C:\Users\default.LAPTOP-EIR1BCFR\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\default.LAPTOP-EIR1BCFR\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
    self.io_loop.start()
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
    self.asyncio_loop.ru

In [67]:
PHASOR_SCALE = 1000
phasor_viewer = napari.Viewer()
phasor_flipped = phasor.copy()
phasor_flipped [:,[0,1]] = phasor_flipped[:,[1,0]]
phasor_flipped[:,0] *= -1
phasor_points_layer = phasor_viewer.add_points(phasor_flipped * PHASOR_SCALE, size=1, properties={'intensity': intensity.ravel()},face_color='intensity',face_colormap='gray')
circle = np.asarray([[0,0.5 * PHASOR_SCALE],[0.5 * PHASOR_SCALE,0.5 * PHASOR_SCALE]])
phasor_shapes_layer = phasor_viewer.add_shapes(circle, shape_type='ellipse', face_color='')

Exception in callback BaseAsyncIOLoop._handle_events(13092, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(13092, 1)>
Traceback (most recent call last):
  File "C:\Users\default.LAPTOP-EIR1BCFR\AppData\Local\Programs\Python\Python39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\zmq\eventloop\zmqstream.py", line 448, in _handle_events
    self._handle_recv()
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\zmq\eventloop\zmqstream.py", line 477, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\zmq\eventloop\zmqstrea

In [61]:
list(napari.utils.colormaps.AVAILABLE_COLORMAPS)

['PiYG',
 'blue',
 'bop blue',
 'bop orange',
 'bop purple',
 'cyan',
 'gist_earth',
 'gray',
 'gray_r',
 'green',
 'hsv',
 'inferno',
 'magenta',
 'magma',
 'plasma',
 'red',
 'turbo',
 'twilight',
 'twilight_shifted',
 'viridis',
 'yellow',
 'label_colormap']

In [11]:
i = tuple(np.array([points_layer.data[0]]).T)
rld_selected, lm_selected = compute_selected(photon_count[i][0])

print(lm_selected.param)



[0.31461743 1.16372    4.068102  ]


C:\Users\default.LAPTOP-EIR1BCFR\AppData\Local\Programs\Python\Python39\lib\collections\__init__.py:593: RuntimeWarning: coroutine 'get_msg' was never awaited
  self.update(iterable, **kwds)


In [12]:
viewer_1 = napari.Viewer()
#current_series_layer = viewer_1.add_image(np.asarray([[[99,99,99],[99,0,0],[2,0,255]],[[0,0,99],[0,255,4],[30,0,0]]]))
viewer_1.add_image(np.asarray([[[.5,1],[.5,.6],[.7,0]],[[0,.2],[0,0],[.5,.8]]]))

<Image layer 'Image' at 0x2ac023fbe20>

Exception in callback BaseAsyncIOLoop._handle_events(4624, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(4624, 1)>
Traceback (most recent call last):
  File "C:\Users\default.LAPTOP-EIR1BCFR\AppData\Local\Programs\Python\Python39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\zmq\eventloop\zmqstream.py", line 448, in _handle_events
    self._handle_recv()
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\zmq\eventloop\zmqstream.py", line 477, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\default.LAPTOP-EIR1BCFR\Documents\flimlib-napari-viewer\venv\lib\site-packages\zmq\eventloop\zmqstream.